In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import GRU
from keras.datasets import imdb
import numpy as np
import pandas as pd
import os

Using TensorFlow backend.


In [2]:
names = ['~', 'stageX', 'stageY', 'headX', 'headY' , 'commandX', 'commandY']
df = pd.read_csv('C:\\Users\\Felix\\Desktop\\毕设\\数据\\20200115_153335\\XYdata-13600-good.txt',sep=',',header=None,names=names,na_values=['-1'], index_col=False)
df1 = pd.read_csv('C:\\Users\\Felix\\Desktop\\毕设\\数据\\20200115_154416\\XYdata-750-6400-good.txt',sep=',',header=None,names=names,na_values=['-1'], index_col=False)

print("Starting file:")
print(df[0:10])

print("Ending file:")
print(df[-10:])

Starting file:
   ~   stageX   stageY    headX    headY  commandX  commandY
0  1  4.92227  2.44792  210.329  195.888  0.008663 -0.002653
1  1  4.91331  2.44404  209.282  194.188  0.010410 -0.001959
2  1  4.91337  2.44187  209.423  189.576  0.013621 -0.001917
3  1  4.92063  2.44000  209.246  189.475  0.014675 -0.001758
4  1  4.93597  2.43820  208.985  191.086  0.014294 -0.001522
5  1  4.95664  2.43661  209.029  191.863  0.013883 -0.001454
6  1  4.97924  2.43573  208.586  193.370  0.012949 -0.001140
7  1  5.00523  2.43429  208.995  196.032  0.011188 -0.001224
8  1  5.03304  2.43332  209.020  198.801  0.009119 -0.001181
9  1  5.05857  2.43253  208.622  200.209  0.007674 -0.000918
Ending file:
       ~   stageX   stageY    headX    headY  commandX  commandY
11228  1 -2.46791  4.14121  224.038  182.363  0.021421 -0.014366
11229  1 -2.44529  4.12766  223.888  181.865  0.021651 -0.014366
11230  1 -2.42037  4.11245  223.555  180.099  0.022690 -0.014233
11231  1 -2.39785  4.09749  224.016  176.

In [3]:
freq=300#伺服控制频率
a11=0.0001777326832096361
a12=0.005308842744342313
a21=0.00534395497810797
a22=-0.00019522174569135787
centerX=208
centerY=208

In [4]:
def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(len(obs)-SEQUENCE_SIZE):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [x for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)

In [5]:
#转浮点
def process(a):
    a['headX'] = a['headX'].astype(float)
    a['stageX'] = a['stageX'].astype(float)
    a['stageY'] = a['stageY'].astype(float)
    a['headY'] = a['headY'].astype(float)
    a['commandX'] = a['commandX'].astype(float)
    a['commandY'] = a['commandY'].astype(float)
    headX = a['headX']
    stageX = a['stageX']
    headY = a['headY']
    stageY = a['stageY']
    commandX = a['commandX']
    commandY = a['commandY']
    
    #算坐标stagetarget和速度headspeed
    headErrorX=centerX-headX
    headErrorY=centerY-headY
    stageErrorX=headErrorX * a11 + a12 * headErrorY
    stageErrorY=headErrorX * a21 + a22 * headErrorY
    stageTargetX=(stageX+stageErrorX).tolist()
    stageTargetY=(stageY+stageErrorY).tolist()
    headSpeedX=(np.diff(stageTargetX)*300).tolist()
    headSpeedY=(np.diff(stageTargetY)*300).tolist()
    
    #创建训练和预测数组
    x_train,x_pred = to_sequences(SEQUENCE_SIZE,stageTargetX)
    y_train,y_pred = to_sequences(SEQUENCE_SIZE,stageTargetY)
    xspeed_train,xspeed_pred = to_sequences(SEQUENCE_SIZE,headSpeedX)
    yspeed_train,yspeed_pred = to_sequences(SEQUENCE_SIZE,headSpeedY)

    #删去位移数组第一行，为了和速度数组同样维度
    x_train = np.delete(x_train,0,0)
    y_train = np.delete(y_train,0,0)
    x_pred = np.delete(x_pred,0,0)
    y_pred = np.delete(y_pred,0,0)

    #换行列
    combine = np.array([x_train,y_train,xspeed_train,yspeed_train])
    combine_pred = np.array([x_pred,y_pred])
    combine = combine.swapaxes(0,1)
    combine_pred = combine_pred.swapaxes(0,1)
    combine = combine.swapaxes(1,2)
    
    return combine,combine_pred

In [6]:
#batch size
SEQUENCE_SIZE = 10

In [30]:
#将多个训练集拼起来，！！需要手动添加！！
train0,pred0 = process(df)
train1,pred1 = process(df1)
train = np.vstack((train0,train1))
pred = np.vstack((pred0,pred1))

In [31]:
print('train:',train.shape)
print('pred:',pred.shape)

train: (16867, 10, 4)
pred: (16867, 2)


In [10]:
from tensorflow.keras.callbacks import EarlyStopping

print('Build model...')
model = Sequential()
model.add(GRU(32, dropout=0.0, input_shape=(None,4), recurrent_dropout=0.0,))
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
print('Train...')

model.fit(train,pred,callbacks=[monitor],verbose=2,epochs=1000)

Build model...




Train...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1000





 - 4s - loss: 0.3587
Epoch 2/1000
 - 3s - loss: 0.0267
Epoch 3/1000
 - 3s - loss: 0.0264
Epoch 4/1000
 - 3s - loss: 0.0155
Epoch 5/1000
 - 3s - loss: 0.0117
Epoch 6/1000
 - 3s - loss: 0.0147
Epoch 7/1000
 - 4s - loss: 0.0148
Epoch 8/1000
 - 4s - loss: 0.0130
Epoch 9/1000
 - 4s - loss: 0.0103
Epoch 10/1000
 - 3s - loss: 0.0106
Epoch 11/1000
 - 4s - loss: 0.0131
Epoch 12/1000
 - 4s - loss: 0.0057
Epoch 13/1000
 - 4s - loss: 0.0126
Epoch 14/1000
 - 4s - loss: 0.0077
Epoch 15/1000
 - 4s - loss: 0.0101
Epoch 16/1000
 - 4s - loss: 0.0072
Epoch 17/1000
 - 4s - loss: 0.0093
Epoch 18/1000
 - 4s - loss: 0.0092
Epoch 19/1000
 - 4s - loss: 0.0070
Epoch 20/1000
 - 5s - loss: 0.0074
Epoch 21/1000
 - 4s - loss: 0.0054
Epoch 22/1000
 - 4s - loss: 0.0068
Epoch 23/1000
 - 4s - loss: 0.0073
Epoch 24/1000
 - 4s - loss: 0.0081
Epoch 25/1000
 - 4s - loss: 0.0090
Epoc

In [11]:
names = ['~', 'stageX', 'stageY', 'headX', 'headY' , 'commandX', 'commandY']
df_test = pd.read_csv('C:\\Users\\Felix\\Desktop\\毕设\\数据\\20200114_221920\\XYdata-3700-13700-爬壁翻车.txt',sep=',',header=None,names=names,na_values=['-1'], index_col=False)

print("Starting file:")
print(df_test[0:10])

print("Ending file:")
print(df_test[-10:])

Starting file:
   ~   stageX   stageY    headX    headY  commandX  commandY
0  1  3.26959  4.41350  228.967  214.815 -0.013823 -0.010212
1  1  3.25408  4.40170  229.229  215.643 -0.013942 -0.010364
2  1  3.23929  4.39112  228.731  215.985 -0.014150 -0.010269
3  1  3.22730  4.38191  228.193  217.854 -0.015165 -0.010042
4  1  3.21601  4.37178  228.341  218.608 -0.015985 -0.010104
5  1  3.20277  4.36195  227.736  221.410 -0.017744 -0.009871
6  1  3.18790  4.35246  230.332  222.146 -0.018932 -0.011122
7  1  3.17102  4.34323  229.376  226.346 -0.021555 -0.011164
8  1  3.15328  4.33215  230.979  224.056 -0.021648 -0.012152
9  1  3.13667  4.32049  231.674  224.597 -0.022156 -0.012979
Ending file:
      ~   stageX   stageY    headX    headY  commandX  commandY
9990  1 -3.90868 -3.43515  215.795  199.919  0.008358 -0.007048
9991  1 -3.89729 -3.44143  215.627  202.019  0.007161 -0.007516
9992  1 -3.88603 -3.44917  218.839  203.393  0.005983 -0.009330
9993  1 -3.87595 -3.45719  221.526  204.561  

In [12]:
train_test,pred_test = process(df_test)

In [34]:
from sklearn import metrics

pred_0 = model.predict(train_test)
score = np.sqrt(metrics.mean_squared_error(pred_0[:,1],pred_test[:,1]))
print("测试集Score (RMSE): {}".format(score))

测试集Score (RMSE): 0.02863434332259947


In [32]:
print(train.shape)
print(pred.shape)

(16867, 10, 4)
(16867, 2)


In [33]:
from sklearn import metrics

pred_1 = model.predict(train)
score = np.sqrt(metrics.mean_squared_error(pred_1[:,0],pred[:,0]))
print("训练集Score (RMSE): {}".format(score))

训练集Score (RMSE): 0.052697741251156116


In [35]:
model.save('C:\\Users\\Felix\\Desktop\\毕设\\训练模型\\model.h5')